###   Random Sampling을 통한 new train dataset 구성

In [ ]:
# Random_sampling code
# 이 코드는 대회에서 데이터로 공유한 code 폴더 내부에 위치해야 합니다.

import sys, random
import csv

def random_positive(path,wr,duplicate_check):
    numbers = list(range(1,301))
    random_num = random.choice(numbers)
    random_dir = "problem"+str(random_num).zfill(3)
    code = []
    tuple_set = []
    for _ in range(2):
        random_file = random.randint(1,150)
        num = 0
        while (random_num,random_file) in duplicate_check and len(numbers)>0:
            random_file = random.randint(1,150)
            num += 1
            if num > 100:
                numbers.remove(random_num)
                random_num = random.choice(numbers)
                num=0
        tuple_set.append((random_num,random_file))
        file_name = path+"/"+random_dir+"/"+random_dir+"_"+str(random_file)+".py"

        with open(file_name,'r') as file:
            code.append("".join(file.readlines()))
    wr.writerow([code[0],code[1],1])
    return tuple_set


def random_negative(path,wr,duplicate_check):
    code = []
    tuple_set = []
    random_num1 = random.randint(1,300)
    random_file1 = random.randint(1,150)
    num1 = 0
    while (random_num1,random_file1) in duplicate_check:
        random_file1 = random.randint(1,150)
        num1 += 1
        if num1>100:
            random_num1 = random.randint(1,300)
            num1=0
    tuple_set.append((random_num1,random_file1))
    
    while(True):
        random_num2 = random.randint(1,300)
        if random_num2 != random_num1:
            break
    random_file2 = random.randint(1,150)
    num2=0
    while (random_num2,random_file2) in duplicate_check:
            random_file2 = random.randint(1,150)
            num2 +=1
            if num2>100:
                while(True):
                    random_num2 = random.randint(1,300)
                    if random_num2 != random_num1:
                        break
    tuple_set.append((random_num2,random_file2))
    
    random_dir1 = "problem"+str(random_num1).zfill(3)
    random_dir2 = "problem"+str(random_num2).zfill(3)
    
    file_name1 = path+"/"+random_dir1+"/"+random_dir1+"_"+str(random_file1)+".py"
    file_name2 = path+"/"+random_dir2+"/"+random_dir2+"_"+str(random_file2)+".py"
    with open(file_name1,'r') as file:
        code.append("".join(file.readlines()))
    with open(file_name2,'r') as file:
        code.append("".join(file.readlines()))
    
    wr.writerow([code[0],code[1],0])
    return tuple_set


if __name__ == "__main__":
    path = str(sys.path[0])
    f = open('new_train_sample_5.csv','w',newline='\n') # 생성된 파일을 아래 코드에서 train dataset으로 불러옵니다.
    wr = csv.writer(f)
    wr.writerow(['code1','code2','similar'])
    row = 0
    random_list = [random.randint(0,1) for r in range(45088)]
    duplicate_check_positive = []
    duplicate_check_negative = []

    while(row<45088): #pair 개수
        if row%100==0: print(row)
        if random_list[row] == 0:
            duplicate_check_positive += random_positive(path,wr,duplicate_check_positive)
        elif random_list[row] == 1:
            duplicate_check_negative += random_negative(path,wr,duplicate_check_negative)
        row +=1

    f.close()

### 모델 구현 및 학습, 추론

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#optimization: AdamW
#model: microsoft/CodeBERT
#learning rate: 2e-5
#epoch: 7
#train_test_split=0.1, seed = 100
#batch_size = 16
#train_sample_size = 45088

In [ ]:
!pip install torch
!pip install transformers
!pip install transformers datasets

In [ ]:
import pandas as pd
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW


*   Load Train dataset

In [ ]:
train = pd.read_csv('new_train_sample_5.csv', encoding = 'utf-8')
df = pd.DataFrame(train)


*   Define Model, tokenizer


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

batch_size = 16
epoch_num = 7
MAX_LEN = 512
learning_rate = 2e-5

* Preprocessing



In [ ]:
# 전처리는 사실상 토큰화하는 것까지 포함하고 있으나 여기서는 편의상 따로 함수를 만들어 진행했습니다.

def preprocess(df,file_name):
  #preprocess_df = df.replace(re.compile('(^import.*|^from.*)',re.MULTILINE),"",regex=True) #import,from 없애기
  preprocess_df = df.replace(re.compile('(#.*)', re.MULTILINE),"",regex=True) #주석 한 줄
  preprocess_df = preprocess_df.replace(re.compile('[\'\"]{3}.*?[\'\"]{3}', re.DOTALL),"",regex=True) #주석 여러줄
  preprocess_df = preprocess_df.replace(re.compile('[\n]{2,}', re.MULTILINE),"\n",regex=True) #다중개행 한번으로
  preprocess_df = preprocess_df.replace(re.compile('[ ]{4}', re.MULTILINE),"\t",regex=True) #tab 변환
  preprocess_df = preprocess_df.replace(re.compile('[ ]{1,3}', re.MULTILINE)," ",regex=True) #공백 여러개 변환
  preprocess_df.to_csv(file_name)
  
def tokenized(examples):
  return tokenizer(examples['code1'],examples['code2'], padding=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [ ]:
preprocess(df,"preprocess.csv")
dataset = load_dataset("csv",data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized,remove_columns=['Unnamed: 0','code1','code2'],batched=True)
encoded_dataset=encoded_dataset.rename_column(original_column_name='similar',new_column_name='labels')

* 데이터 나누기

In [ ]:
encoded_dataset = encoded_dataset.train_test_split(0.1,seed=100)

* optimizers 설정

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8, 
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
args = TrainingArguments("test", save_strategy="epoch",evaluation_strategy="epoch",logging_strategy="epoch", 
                         learning_rate=learning_rate,per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,num_train_epochs=epoch_num,weight_decay=0.01,
                         do_train=True,do_eval=True,metric_for_best_model="accuracy",load_best_model_at_end=True)

trainer = Trainer(model,args,train_dataset=encoded_dataset['train'],eval_dataset=encoded_dataset['test'],
                  tokenizer=tokenizer, compute_metrics=compute_metrics, data_collator=data_collator,
                  optimizers=optimizers)


In [ ]:
gc.collect()
torch.cuda.empty_cache()

* 학습

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 40579
  Num Epochs = 7
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 17759


Epoch,Training Loss,Validation Loss,Accuracy
1,0.269900,0.145242,0.952096
2,0.146900,0.158933,0.956310
3,0.110700,0.203906,0.956088
4,0.083300,0.218258,0.956531
5,0.060300,0.188926,0.969173
6,0.046500,0.172191,0.970725
7,0.040300,0.194919,0.969173


***** Running Evaluation *****
  Num examples = 4509
  Batch size = 16
Saving model checkpoint to test/checkpoint-2537
Configuration saved in test/checkpoint-2537/config.json
Model weights saved in test/checkpoint-2537/pytorch_model.bin
tokenizer config file saved in test/checkpoint-2537/tokenizer_config.json
Special tokens file saved in test/checkpoint-2537/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4509
  Batch size = 16
Saving model checkpoint to test/checkpoint-5074
Configuration saved in test/checkpoint-5074/config.json
Model weights saved in test/checkpoint-5074/pytorch_model.bin
tokenizer config file saved in test/checkpoint-5074/tokenizer_config.json
Special tokens file saved in test/checkpoint-5074/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4509
  Batch size = 16
Saving model checkpoint to test/checkpoint-7611
Configuration saved in test/checkpoint-7611/config.json
Model weights saved in test/checkpoint-7611/pytorch_mod

TrainOutput(global_step=17759, training_loss=0.10828670611473627, metrics={'train_runtime': 15893.4995, 'train_samples_per_second': 17.872, 'train_steps_per_second': 1.117, 'total_flos': 7.473748460817408e+16, 'train_loss': 0.10828670611473627, 'epoch': 7.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 4509
  Batch size = 16


{'epoch': 7.0,
 'eval_accuracy': 0.9707252162341983,
 'eval_loss': 0.1721908301115036,
 'eval_runtime': 80.8684,
 'eval_samples_per_second': 55.757,
 'eval_steps_per_second': 3.487}

In [ ]:
TEST = "test.csv"
SAMPLE = "sample_submission.csv"

test = pd.read_csv('test.csv', encoding = 'utf-8')
df_test = pd.DataFrame(test)

preprocess(df_test,"preprocess_test.csv")

test_dataset = load_dataset("csv",data_files = "preprocess_test.csv")['train']
test_dataset = test_dataset.map(tokenized,remove_columns=['code1','code2'])

predictions = trainer.predict(test_dataset)

df = pd.read_csv(SAMPLE)
df['similar'] = np.argmax(predictions.predictions,axis=-1)
df.to_csv('/content/drive/MyDrive/submission.csv',index=False)